In [107]:
# Dependencies
import numpy as np
import pandas as pd
import plotly.colors as pc
import plotly.express as px
import plotly.graph_objects as go


In [108]:
#Read csv file
disaster_df = pd.read_csv('./Resources/Climate-related_Disasters_Frequency.csv')

disaster_df.head()

,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,...,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022
0,1,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN
1,2,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,4.0,2.0,1.0,4.0,1.0,3.0,6.0,5.0,2.0,5.0
3,4,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,1.0,NaN,4.0,NaN,2.0,1.0,1.0,1.0,1.0,1.0
4,5,"Afghanistan, Islamic Rep. of",AF,AFG,"Climate related disasters frequency, Number of...",Number of,"The Emergency Events Database (EM-DAT) , Centr...",ECCD,Climate Related Disasters Frequency,"Environment, Climate Change, Climate Indicator...",...,NaN,1.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN


In [109]:
# Split off indicator from uneeded text
disaster_df['Indicator'] = disaster_df['Indicator'].str.split(':').str[1]

In [110]:
# Select columns for graphs
disaster_df = pd.concat([disaster_df['Country'], disaster_df['Indicator'],disaster_df.loc[:, 'F1980':'F2022']], axis=1)

# Find out how my disaster indicators exist - 7
disaster_df['Indicator'].unique()


array([' Drought', ' Extreme temperature', ' Flood', ' Landslide',
       ' Storm', ' TOTAL', ' Wildfire'], dtype=object)

In [111]:
# creaet summary table of counts for indicators
indicator_df = disaster_df.groupby('Indicator').count().drop(columns=['Country']
                        ).drop(' TOTAL').rename(index={' Extreme temperature': 'Extreme Temp'})
# remove country column
indicator_df
# indicator_df.to_excel('indicator.xlsx')


,F1980,F1981,F1982,F1983,F1984,F1985,F1986,F1987,F1988,F1989,...,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022
Indicator,,,,,,,,,,,,,,,,,,,,,
Drought,24,18,6,25,6,3,4,15,17,7,...,9,19,27,14,12,17,15,12,16,22
Extreme Temp,2,1,2,2,1,6,2,5,4,3,...,13,15,10,11,9,25,14,5,3,12
Flood,25,28,32,31,28,32,35,40,50,36,...,81,66,82,75,66,72,83,89,93,78
Landslide,3,7,9,12,6,3,5,10,12,10,...,7,10,13,9,17,10,20,16,10,12
Storm,23,25,23,27,28,30,28,32,27,32,...,38,35,48,36,65,43,43,58,55,50
Wildfire,1,2,2,10,5,9,4,7,0,2,...,5,3,8,7,11,8,12,8,14,11


In [112]:


# indicator_df = indicator_df.transpose()

# Create a list of indicators to plot (excluding the 'TOTAL' row)
indicators = indicator_df.index[:-1]

# Create a list of years from the columns
years = indicator_df.columns.tolist()

# Create a list of line traces for each indicator
traces = []
for indicator in indicators:
    trace = go.Scatter(
        x=years,
        y=indicator_df.loc[indicator],
        name=indicator
    )
    traces.append(trace)

# Create the layout
layout = go.Layout(
    title='Indicator Trends',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Count')
)

# Create the figure
fig = go.Figure(data=traces, layout=layout)

# Show the figure
fig.show()

In [113]:
# Create a list of indicators
indicators = indicator_df.index

# Get a color sequence for the indicators
color_sequence = pc.qualitative.Plotly

# Create a line chart for each indicator
for i, indicator in enumerate(indicators):
    x = indicator_df.columns.values[1:]  # Exclude the first column (Indicator)
    y = indicator_df.loc[indicator].values[1:]  # Exclude the first column (Indicator)
    
    # Create a trace for the line chart with a different color for each indicator
    trace = go.Scatter(
        x=x,
        y=y,
        mode='lines',
        name=indicator,
        line=dict(color=color_sequence[i % len(color_sequence)])
    )
    
    # Compute the trendline using linear regression
    trendline_coef = np.polyfit(range(len(x)), y, 1)
    trendline_y = np.polyval(trendline_coef, range(len(x)))
    
    # Create a trace for the trendline
    trendline_trace = go.Scatter(
        x=x,
        y=trendline_y,
        mode='lines',
        name=f'Trendline',
        line=dict(color='black')
    )
    
    # Create the layout for the line chart
    layout = go.Layout(
        title=f'Time Series with Trendline for{indicator}',
        xaxis=dict(title='Year'),
        yaxis=dict(title='Count'),
        height=400,
        width=800
    )
    
    # Create the figure for the line chart
    fig = go.Figure(data=[trace, trendline_trace], layout=layout)
    
    # Show the figure
    fig.show()